In [68]:
#用于1车道1交通灯和 pytorch测试
#作者lukeliuli@163.com
# -*- coding: utf-8 -*-
"""
@author: lukeliuli@163.com
"""
%cd /root/myPrjs/SUMONBDT

import numpy as np
import os
import sys

import random


if 'SUMO_HOME' in os.environ:
     tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
     sys.path.append(tools)
else:
     sys.exit("please declare environment variable 'SUMO_HOME'")
     
import libsumo

path = os.getcwd()
path2 = path+"\sumoCfgs\my1Lane1TlsVeh3-server.sumocfg"#window
path2 = path+"/sumoCfgs1/my1Lane1TlsVeh3-server.sumocfg"#linux
sumoBinary = "sumo"
libsumo.start([sumoBinary, "-c", path2])


#定义模拟过程中动态可调整参数
params ={}
params["GreenTime"] = 15
params["redTime"] = 15
params["ObjVehPosInXY"] = [-100,0]
params["MaxAcc"] = 1
params["DrvResTime"] = 0.5
params["addedVeh"] = 2+1
    
    
numSim = 1
nextTLSID = "2"
veh = "00003"
for i in range(numSim):
    print("##################start sim %d##################" %i)
    libsumo.start([sumoBinary, "-c", path2])
    stepNum = 0
    requireStop = 0
    passTLS = 0
    
    ##修改出发位置，随机参数
    X =  params["ObjVehPosInXY"][0]+min(random.random()*100,90)#注意坐标的原点
    Y =  params["ObjVehPosInXY"][1]
    libsumo.vehicle.moveToXY(veh,"e1to2", 0, X, Y, -1073741824.0, 1)
    
    
    ####修改绿灯时间，随机参数
    """
    greenTime =  params["GreenTime"]#注意坐标的原点
    greenTime = max(random.random()*greenTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    greenDurTime = lgc1.phases[1].duration
    yellowDurTime = lgc1.phases[2].duration
    redDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = greenTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
    """
    
    ####修改逻辑和红灯时间，随机参数
    redTime =  params["redTime"]#注意坐标的原点
    redTime = max(random.random()*redTime,0.1)
    allProgramLogics = libsumo.trafficlight.getAllProgramLogics(nextTLSID)
    lgc1 = allProgramLogics[0]
    yellowDurTime = lgc1.phases[0].duration
    
    lgc1.phases[1].state = 'r'
    redDurTime = lgc1.phases[1].duration
    
    yellowDurTime = lgc1.phases[2].duration
    
    lgc1.phases[3].state = 'G'
    greenDurTime = lgc1.phases[3].duration
    
    lgc1.phases[1].duration = redTime
    libsumo.trafficlight.setProgramLogic(nextTLSID,lgc1) 
    phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
   
    #print(X,greenTime,greenDurTime,yellowDurTime,redDurTime)
    print(lgc1)
    
    
    while requireStop == 0:
        libsumo.simulationStep()
        stepNum += 1
        
        ####https://sumo.dlr.de/daily/pydoc/traci._vehicle.html
        nextTLSID = "2"
        TLSPos = 0
        states = libsumo.trafficlight.getRedYellowGreenState(nextTLSID)
        timeT = libsumo.simulation.getCurrentTime()  # 当前时间
        nextSwitch = libsumo.trafficlight.getNextSwitch(nextTLSID)
        phaseName = libsumo.trafficlight.getPhaseName(nextTLSID)
        phase = libsumo.trafficlight.getPhase(nextTLSID)
        phaseDur = libsumo.trafficlight.getPhaseDuration(nextTLSID)
        curTime = timeT/1000
        phaseLeftTime = nextSwitch - curTime
        
        ####
        veh = "00003"
        tlsList = libsumo.vehicle.getNextTLS(veh)
        #tlsID_T, tlsIndex, dist, state = tlsList[0]
       
        
        vehPos = libsumo.vehicle.getPosition(veh)[0]
        speed = libsumo.vehicle.getSpeed(veh)
        vehMaxSpeed = libsumo.vehicle.getMaxSpeed(veh)

        laneID = libsumo.vehicle.getLaneID(veh)
        #meanSpeed = libsumo.lane.getLastStepMeanSpeed(laneID)
        laneMaxSpeed = libsumo.lane.getMaxSpeed(laneID)

        vehMaxSpeed = min(vehMaxSpeed, laneMaxSpeed)
        
        dist = TLSPos-vehPos
        arrivalTime1 = min(100,dist / (speed + 0.001))
        arrivalTime2 = min(100,dist / vehMaxSpeed)
        #arrivalTime3 = min(100,dist / (meanSpeed+0.001))

        #predictMeanSpeed1min = laneMaxSpeed  #1分钟前绿灯时,当前场景下目标车道的平均速度
        #arrivalTime4 = min(dist / predictMeanSpeed1min,100)
    
    
        
        if states == "r" and  dist >0:
            passTLS = 0
            

            
        if states == "G" and  dist <=0:
            passTLS = 1
        if states == "r" and  dist <=0:
            passTLS = 1
        
            
            
        if timeT/1000 > 30 or states == "r" or dist<=0:  # if time is over 30 second stoping the simulation
            requireStop = 1
            
       
        print("#####tls state: %s,duration:%.3f,running time(ms):%d,passTLS:%d,dist:%d" %(states,phaseDur,timeT,passTLS,dist))

    libsumo.close()
    


/root/myPrjs/SUMONBDT
Loading configuration ... done.
##################start sim 0##################
Loading configuration ... done.
Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=0.01, state='y', minDur=0.01, maxDur=0.01, next=()), Phase(duration=6.808030733965479, state='r', minDur=20.0, maxDur=20.0, next=()), Phase(duration=0.01, state='y', minDur=0.01, maxDur=0.01, next=()), Phase(duration=20.0, state='G', minDur=20.0, maxDur=20.0, next=())), subParameter={})
#####tls state: y,duration:0.010,running time(ms):10,passTLS:0,dist:98
#####tls state: r,duration:6.808,running time(ms):20,passTLS:0,dist:98
